##### Importar Dependencias

In [1]:
from os import listdir
from scr.util import *

##### Definir el documento de entrada del inicio y fin de los lanzamientos y los nombres de las carpetas de salida

In [3]:
# data_doc_path = "./data/SCUvsIND.txt"

# default_videos_output_path = "./output/Clips/"
# dein_videos_output_path = "./output/Clips_dein/"
# detected_images_output_path = "./output/BallDetected_yolo/"
# not_detected_images_output_path = "./output/BallNotDetected_yolo/"

# default_name = "SCUvsIND_02"
# video = "./data/video/SCUvsIND_02.mts"

# model_path = "../YOLOv8/yolov8m.pt"

default_videos_output_path = "./output/Clips/"
dein_videos_output_path = "./output/Clips_dein/"
detected_images_output_path = "./output/BallDetected_yolo/"
not_detected_images_output_path = "./output/BallNotDetected_yolo/"

# INPUT
data_doc_path = "./data/MTZvsGRM.txt"
video = "./data/video/MTZvsGRM_01.mts"
default_name = video.split("/")[-1].split(".")[0]

model_path = "../YOLOv8/yolov8m.pt"

# MASTER
MAX_VIDEOS_PROCS = 10000000

#### Recortar los videos

In [7]:
count = 1

data = load_doc_data(data_doc_path)

for M, N in load_doc_data(data_doc_path):
    trim_video(  # Margen de Error M - 5  -->  N + 5
        video,
        default_videos_output_path
        + default_name
        + format_name_default_output_video(count),
        M - 5,
        N + 5,
        1,
    )
    count += 1
    if count > MAX_VIDEOS_PROCS:
        break


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


KeyboardInterrupt: 

#### Desentrelazar los clips

In [ ]:
for clip in listdir(default_videos_output_path):
    print(format_name_dein_output_video(clip))
    output_name = dein_videos_output_path + format_name_dein_output_video(clip)
    deinterlace_video(default_videos_output_path + clip, output_name)
    

#### Cargar el modelo y dependencias

In [4]:
import cv2
import torch
from ultralytics import YOLO

model = YOLO(model_path)
print(torch.cuda.is_available())

True


In [6]:
clips = listdir(dein_videos_output_path)
clips.sort()
count = 1

new_path = "./output/Plot/"

for clip in clips:
    cap = cv2.VideoCapture(dein_videos_output_path + clip)

    while True:
        ret, frame = cap.read()

        if not ret:
            break
        result = model(frame)

        output_img = not_detected_images_output_path
        tmp = format_name_output_img(clip, count)

        if 32 in result[0].boxes.cls:
            output_img = detected_images_output_path
            index = result[0].boxes.cls.tolist().index(32)
            x, y, x1, y1 = result[0].boxes.xyxy[index]
            copy = frame.copy()
            cv2.rectangle(copy, (int(x), int(y)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imwrite(new_path + tmp, copy)
            cv2.imwrite(output_img + tmp, frame)
            continue
        else:
            cv2.imwrite(output_img + tmp, frame)
        count += 1


0: 384x640 4 persons, 1 baseball bat, 1 baseball glove, 15.7ms
Speed: 5.1ms preprocess, 15.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 baseball bat, 1 baseball glove, 15.7ms
Speed: 1.8ms preprocess, 15.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 baseball bat, 15.7ms
Speed: 1.8ms preprocess, 15.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 baseball bat, 1 baseball glove, 15.7ms
Speed: 2.0ms preprocess, 15.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 baseball bat, 15.7ms
Speed: 1.2ms preprocess, 15.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2 baseball bats, 1 baseball glove, 15.6ms
Speed: 1.2ms preprocess, 15.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 baseball bat, 1 baseball glove, 15.6ms
Speed